**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install umap-learn
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import NearestNeighbors
from umap import UMAP

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numbers

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("iris.csv"), directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
cmap = 'viridis'

def plot_data(data, colors, alpha=1.0, ax=None,
              edgecolors=None, cmap=cmap, s=50):
    if ax is None:
        ax = plt.gca()
    
    ax.scatter(data[:, 0], data[:, 1], s=s,
               c=colors, edgecolors=edgecolors,
               alpha=alpha, cmap=cmap)

    ax.grid(ls='--')
    ax.set_axisbelow(True)
    ax.set_xlabel("$d_1$")
    ax.set_ylabel("$d_2$")
    
def make_legend(class_names, colors=None, cmap=cmap,
                ax=None, alpha=1.0, num_colors=None):
    if ax is None:
        ax = plt.gca()
        
    if isinstance(alpha, numbers.Number):
        alpha_seq = (alpha for i in range(len(class_names)))
    else:
        alpha_seq = alpha
    
    if num_colors is None:
        num_colors = len(class_names)
    
    if colors is None:
        colors = range(len(class_names))
        
    cm = plt.get_cmap(cmap, num_colors)

    legend_handles = []
    for ic, cn, al in zip(colors, class_names, alpha_seq):
        legend_handles.append(
            mpatches.Patch(color=cm(ic), label=cn, alpha=al)
        )

    ax.legend(handles=legend_handles)

## Klasifikácia pomocou KNN: Ilustrácia

Tento notebook grafickou formou ilustruje, ako funguje metóda $k$ najbližších susedov ($k$ nearest neighbours; KNN). Notebook nemá slúžiť ako referenčný príklad praktického použitia metódy KNN (na to je k dispozícii iný notebook).

### Dátová množina

Pre potreby tejto ilustrácie budeme pracovať s dobre známou dátovou množinou Iris, ktorá obsahuje merania veľkostí okvetných a kališných lístkov pre 3 rôzne druhy kosatcov: setosa, virginica a versicolor. Úlohou je vytvoriť klasifikátor, ktorý by bol schopný od seba tieto 3 rozličné triedy kosatcov rozoznať.

V prvom kroku načítame pomocou balíčka `pandas` dáta z CSV súboru:



In [ ]:
df = pd.read_csv("data/iris.csv")
df.head()

Ako vidno, naše dáta sú 4-rozmerné. Keďže tento ilustračný príklad bude do veľkej miery založený na grafických vizualizáciách, znížime rozmer dáta na 2 (pomocou metódy UMAP, ale tým sa teraz netrápme) predtým než s nimi budeme realizovať ďalšie operácie:



In [ ]:
data_raw = df.iloc[:, :-1]
umap = UMAP(spread=20.0)
data = umap.fit_transform(data_raw)

Zároveň extrahujeme posledný stĺpec, ktorý obsahuje názvy druhov kosatcov. Názvy majú podobu textových reťazcov. Aby sme si prácu s dátami v ďalších fázach uľahčili, priradíme každému druhu kosatca číselný identifikátor a textové reťazce nahradíme číslami:



In [ ]:
str_labels = df[['species']].values
ordenc = OrdinalEncoder(dtype='int')
num_labels = ordenc.fit_transform(str_labels).flatten()
class_names = ordenc.categories_[0]

Po týchto úvodných krokoch sme pripravení dáta vizualizovať v 2-rozmernom bodovom grafe:



In [ ]:
plot_data(data, num_labels)
make_legend(class_names)
plt.savefig('output/knn_algo_data.pdf', bbox_inches='tight', pad_inches=0)

### $k$ najbližších susedov

Myšlienka metódy $k$ najbližších susedov je veľmi jednoduchá. Kedykoľvek dostaneme nový bod, nazrieme späť do dátovej množiny a nájdeme $k$ bodov, ktoré sú novému bodu najbližšie (t.j. jeho $k$ najbližších susedov vo vstupnom priestore). Trieda nového bodu sa potom určí hlasovaním medzi týmito jeho najbližšími susedmi.

Aby sme tento princíp ilustrovali konkrétnejšie, vyberme si teraz nový bod a jeho pozíciu vizualizujme čiernym krížikom:



In [ ]:
point = [-8, -6]
plot_data(data, num_labels)
plt.scatter(point[0], point[1], marker='x', s=75, linewidth=3, c='k')
plt.savefig('output/knn_algo_point.pdf', bbox_inches='tight', pad_inches=0)

Všimnite si, že predtým, ako sme získali tento nový bod, s dátovou množinou sme nič nerobili. Nepredspracovali sme ju, nepoužili sme ju na ladenie parametrov modelu – len sme si ju odložilo na neskoršie použitie. KNN sa z tohto dôvodu niekedy označuje ako *lenivá*  alebo *neparametrická*  metóda – tak sa vo všeobecnosti nazývajú metódy tohto typu.

V každom prípade, nájdime teraz 3 najbližších susedov nášho pôvodného bodu a zvýraznime ich:



In [ ]:
knn = NearestNeighbors(n_neighbors=3).fit(data)
dist, ind = knn.kneighbors([point])
neigh_colors = num_labels[ind[0]]

plot_data(data, num_labels, alpha=0.4)
plt.scatter(point[0], point[1], marker='x', s=75, linewidth=3, c='k')
plt.scatter(data[ind[0], 0], data[ind[0], 1], s=90,
            c=num_labels[ind[0]], cmap=cmap,
            edgecolors='k', linewidths=1.5,
            vmin=0, vmax=len(class_names))
plt.savefig('output/knn_algo_neighbours.pdf', bbox_inches='tight', pad_inches=0)

Trieda nového bodu sa určí hlasovaním: bod budeme klasifikovať do triedy, ktorá sa medzi susedmi najčastejšie vyskytuje.



In [ ]:
point_color = np.bincount(neigh_colors).argmax()
plot_data(data, num_labels, alpha=0.4, edgecolors=None)
plt.scatter([point[0]], [point[1]], marker='x', s=75, linewidth=3,
            c=[point_color], vmin=0, vmax=len(class_names))
plt.savefig('output/knn_algo_class.pdf', bbox_inches='tight', pad_inches=0)